In [89]:
import pandas as pd
import regex as re
import numpy as np
import json

In [90]:
#df1 = pd.read_csv('laptop_overview_df.csv')
#df2 = pd.read_json('laptop_detail_df.json')
#df=pd.merge(df1, df2, how='inner', on='link')
#df1.shape, df2.shape, df.shape
#df=df.apply(lambda x: x.str.replace('\u200e', ''), axis=1)
#df.to_csv('raw_data_after_merge.csv', index=False)

# input dữ liệu

In [91]:
df=pd.read_csv('raw_data_after_merge.csv')

In [92]:
df.shape

(2106, 79)

# Tiền xử lý

In [93]:
for col in df.columns:
    try:
        df[col]=df[col].str.lower()
    except:
        pass

In [94]:
def nan_1_time_cate(serie):
    valuecounts=serie.value_counts()
    unkeep=list(valuecounts[valuecounts==1].index)
    return serie.replace(unkeep, np.nan)

## cpu

In [95]:
df['Processor']=df['Processor'].str.replace(r'_', ' ')

In [96]:
dfsub=df[['name', 'description', 'Processor']]
cpu_extracted_by_regex=[]
cpu_regex=r'((mediatek|ryzen|celeron|xeon|intel|pentiumn|core|amd).{0,30}?(\w+\d{4}\w*|\w*\d{4}\w+)|snapdragon 7c( gen 2){0,1})'
k=0
for i in range(dfsub.shape[0]):
    found=re.search(
        cpu_regex, 
        dfsub.iloc[i]['name'],
        flags=re.IGNORECASE
    )
    if found:
        cpu_extracted_by_regex.append(found.group())
        #print(dfsub.iloc[i, 0].strip())
        #print(found.group())
    else:
        try:
            found2=re.search(
                cpu_regex, 
                dfsub.iloc[i]['description'],
                flags=re.IGNORECASE
            )
        except TypeError:
            pass
            found2=False
        finally:
            if found2:
                #print(dfsub.iloc[i, 0].strip())
                #print(found2.group())
                k+=1
                cpu_extracted_by_regex.append(found2.group())
            else:
                cpu_extracted_by_regex.append(np.nan)
                #print(df.iloc[i]['official link'])
                #print(dfsub.iloc[i]['name'])
                #print(dfsub.iloc[i]['Processor'])

In [97]:
df['Processor']=cpu_extracted_by_regex

In [98]:
df['Processor']=df['Processor'].str.replace('_', ' ')
#df['Processor']=df['Processor'].str.replace(r'core|pentinum|celeron', 'intel', regex=True)
#df['Processor']=df['Processor'].str.replace(r'ryzen', 'amd', regex=True)

In [99]:
cpu_df=pd.read_csv('cpu_rank.csv')
cpu_df_list=list(zip(cpu_df['CPU name'], cpu_df['CPU rank']))
cpu_df_list=[list(x) for x in cpu_df_list]
for i in range(len(cpu_df_list)):
    cpu_df_list[i].append(cpu_df_list[i][0].split()[-1])
cpu_df_list

[['amd 3015ce', 2199, '3015ce'],
 ['amd 3015e', 1911, '3015e'],
 ['amd 3020e', 2000, '3020e'],
 ['amd 4700s', 405, '4700s'],
 ['amd a10 micro 6700t apu', 2617, 'apu'],
 ['amd a10 pro 7350b apu', 2266, 'apu'],
 ['amd a10 pro 7800b apu', 1720, 'apu'],
 ['amd a10 pro 7850b apu', 1643, 'apu'],
 ['amd a10 4600m apu', 2300, 'apu'],
 ['amd a10 4655m apu', 2474, 'apu'],
 ['amd a10 4657m apu', 2401, 'apu'],
 ['amd a10 5700 apu', 1882, 'apu'],
 ['amd a10 5745m apu', 2410, 'apu'],
 ['amd a10 5750m apu', 2274, 'apu'],
 ['amd a10 5757m apu', 2230, 'apu'],
 ['amd a10 5800b apu', 1818, 'apu'],
 ['amd a10 5800k apu', 1806, 'apu'],
 ['amd a10 6700 apu', 1754, 'apu'],
 ['amd a10 6700t apu', 2071, 'apu'],
 ['amd a10 6790k apu', 1759, 'apu'],
 ['amd a10 6800b apu', 1815, 'apu'],
 ['amd a10 6800k apu', 1734, 'apu'],
 ['amd a10 7300 apu', 2418, 'apu'],
 ['amd a10 7400p', 2178, '7400p'],
 ['amd a10 7700k apu', 1738, 'apu'],
 ['amd a10 7800 apu', 1733, 'apu'],
 ['amd a10 7850k apu', 1660, 'apu'],
 ['amd a10 7

In [100]:
#truong hop loi
#Intel Xeon E3-1246 v3
#amd ryzen 5 5500u vs amd ryzen 5 pro 5500u
def transform_cpu_name_and_rank(row):
    processor = row['Processor']
    if pd.notna(processor):
        """
        brand=re.findall(r'intel|amd|snapdragon|mediatek', processor)[0]
        model=re.findall('\w+$', processor)[0]
        found=False
        for x in cpu_df_list:
            if brand==x[2] and model==x[3]:
                row['Processor']=x[0]
                row['Processor rank']=x[1]
                found=True
                break
            if(found==False):
                row['Processor']=np.nan
                row['Processor rank']=np.nan
        return row
        """
        model = re.findall("\w+$", processor)[0]
        match1 = []
        for i, x in enumerate(cpu_df_list):
            if x[2] == model:
                match1.append(i)
        if len(match1) >= 2:
            found = (-1, -1)
            processor_wbw = processor.split()[:-1]
            for x in match1:
                f = 0
                for word in processor_wbw:
                    if word in cpu_df_list[x][0]:
                        f += 1
                if f > found[0]:
                    found = (f, x)
            pos = found[1]
        elif len(match1)==1:
            pos = match1[0]
        else:
            row["Processor"] = np.nan
            row["Processor rank"] = np.nan
            #print(row["Processor"])
            return row
        row["Processor"] = cpu_df_list[pos][0]
        row["Processor rank"] = cpu_df_list[pos][1]
    return row


df=df.apply(transform_cpu_name_and_rank, axis=1)

In [101]:
len(df['Processor'].value_counts())

178

In [44]:
df['Processor'].apply(lambda x: re.findall(r'^(\w*)\s', x)[0] if pd.notna(x) else x).value_counts()

intel         1162
amd            311
snapdragon       5
mediatek         3
Name: Processor, dtype: int64

## card

In [45]:
with open('card-performance.json', 'r') as rf:
    card_perf_df=json.load(rf) 
card_list=[x[0] for x in card_perf_df]

In [46]:
from Levenshtein import ratio
"""
def LCS(s, t):
    n = len(s)
    m = len(t)
    s=' '+s
    t=' '+t

    f = [[0 for i in range(m + 1)] for j in range(n+1)]
     
    for i in range(1,n + 1):
        for j in range(1,m + 1):
            if(s[i - 1] == t[j - 1]):
                f[i][j] = f[i - 1][j - 1] + 1
            else:
                f[i][j] = max(f[i-1][j], f[i][j-1])
    return (f[n][m]+n+m)/(n+m)
"""
def find_relative_string(s, slist):
    found=[-1, -1] #max value,position
    for i in range(len(slist)):
        acc=ratio(s, slist[i])
        if found[0]<acc:
            found[0]=acc
            found[1]=i
    return found


def transform_graphic(row):
    card = row['Graphics Coprocessor']
    if pd.notna(card):
        card=card.lower()
        acc, pos=find_relative_string(card, card_list)
        if acc>0.5:
            #print(card)
            #print(card_perf_df[pos][0])
            row["Graphics Coprocessor"] = card_perf_df[pos][0]
            row["Graphics Coprocessor perf"] = card_perf_df[pos][1]
        else:
            row["Graphics Coprocessor"] = np.nan
            row["Graphics Coprocessor perf"] = np.nan
    return row


df=df.apply(transform_graphic, axis=1)

## weight

In [47]:
df['weight']=df['Item Weight'].str.split().str[0]
df['weight']= df['weight'].astype(float)
df['weight'] *= 0.45359237

## Hard Drive Type/size

In [48]:
df['Hard Drive Type']=df['Hard Drive'].str.split().str[2]
df['HD Size']= df['Hard Drive'].str.extract('(\d+)').fillna(np.nan).astype(float)
df['HD_Type']=df['Hard Drive'].str.split().str[1] 
df['HD_Type']=df['HD_Type'].apply({'gb':1,'tb':1024}.get)
df['Hard Drive Size'] = df['HD Size'] * df['HD_Type']

In [49]:
df['Hard Drive Type']=nan_1_time_cate(df['Hard Drive Type'])
drop_index = df[(df['Hard Drive Type'].apply(str).apply(str.isdigit))].index
df.drop(drop_index, inplace=True)

In [50]:
df['Hard Drive Type']=df['Hard Drive Type'].replace('solid', 'ssd')

In [51]:
df['Hard Drive Type']=df['Hard Drive Type'].replace(regex=r'.*([^ssd][^hdd][^hybrid]).*', value='other')

In [52]:
df['Hard Drive Type'].value_counts()

ssd       1064
other       78
hdd         34
hybrid       6
Name: Hard Drive Type, dtype: int64

## screen size

In [53]:
df['Screen Size']=df['Standing screen display size'].str.split().str[0]
df['Screen Size']=nan_1_time_cate(df['Screen Size'])

## memory size/type/speed

In [54]:
df['RAM']=df['RAM'].replace({'lpddr 5':'lpddr5'}, regex=True)
df['Memory Size'] =  df['RAM'].str.extract('(\d+)').fillna(np.nan)
df['Memory Type'] =df['RAM'].str.split().str[-1]
df['Memory Type']=df['Memory Type'].replace(['gb', 'tb'], np.nan)

In [55]:
nan_1_time_cate(df['Memory Type']).value_counts()

ddr4       994
lpddr4x    336
lpddr4     115
ddr5        59
ddr3        58
sdram       38
lpddr5      28
lpddr3      21
so-dimm     14
ddr3l       13
ddr         11
dram         3
ddr2         3
Name: Memory Type, dtype: int64

In [56]:
df['Memory Speed']=df['Memory Speed'].replace({' ghz':'',' mhz':'/1000'}, regex=True).dropna().apply(pd.eval)
df['Memory Speed']=df['Memory Speed'].apply(lambda x: x/1000 if x>1000 else x)

## customer review

In [57]:
df['Customer Reviews']=df['Customer Reviews'].str.split().str[0]

## price

In [58]:
df['old_price']=df['old_price'].str.extract('(\d+.\d+)')
df['old_price']=df['old_price'].str.replace(',', '')
df['price']=df['price'].str.extract('(\d+.\d+)')
df['price']=df['price'].str.replace(',', '')

## laptop type

In [59]:
def extract_type(row):
    if(pd.notna(row)):
        x = re.findall(r'traditional|2 in 1', row)
        if x:
            return x[0]

df['Laptop type']=df['Best Sellers Rank'].apply(extract_type)

In [60]:
df['Laptop type'].value_counts()

traditional    1749
2 in 1          135
Name: Laptop type, dtype: int64

## best seller rank

In [61]:
df['Best Sellers Rank']=df['Best Sellers Rank'].str.extract(r'^#([\d,]+)')
df['Best Sellers Rank']=df['Best Sellers Rank'].str.replace(',', '')
df['Best Sellers Rank']=df['Best Sellers Rank'].astype(np.float64)

In [62]:
sellerrank=df['Best Sellers Rank'].dropna().tolist()
sellerrank.sort()
sellerrank_dict={}
dem=0
pre=-1
for x in sellerrank:
    if x > pre: 
        dem+=1
        sellerrank_dict[x]=dem
    if(pd.notna(x)):
        pre=x


In [63]:
df['Best Sellers Rank']=df['Best Sellers Rank'].replace(sellerrank_dict)

## Date First Available

In [64]:
df['Date First Available'] = pd.to_datetime(df['Date First Available'])

## operating system

In [65]:
df['Operating System']=df['Operating System'].str.replace(r'.*(windows|win).*', 'windows', regex=True)
df['Operating System']=df['Operating System'].str.replace(r'.*chrome.*', 'chrome os', regex=True)
df['Operating System']=df['Operating System'].str.replace(r'.*[^chrome os][^windows].*', '#', regex=True)
df['Operating System']=df['Operating System'].replace('#', np.nan)

## brand

In [66]:
possible_brand=list(df['Brand'].dropna().unique())
possible_brand=[x for x in possible_brand if x!='intel']

In [67]:
def transfom_brand(row):
    for brand in possible_brand:
        if brand in str(row['name']) or brand in str(row['Brand']):
            row['Brand']=brand
            break
    if(pd.isna(row['Brand'])):
        row['Brand']=re.findall(r'^[\d\s]*(\w+)', row['name'])[0]
    return row
df=df.apply(transfom_brand, axis=1)

In [68]:
df['Brand']=df['Brand'].replace('hewlett packard', 'hp')
df['Brand']=df['Brand'].replace('intel', np.nan)

In [69]:
df['Brand']=nan_1_time_cate(df['Brand'])

In [70]:
df['Brand'].value_counts()

lenovo       576
hp           409
acer         361
dell         238
asus         237
msi          131
samsung       48
lg            28
microsoft     18
razer         17
gigabyte      11
toughbook      6
dynabook       3
hyundai        3
xpg            2
Name: Brand, dtype: int64

## laptop purpose

In [71]:
def extract_purpose(row):
    x = re.findall(r'gaming', row)
    if x:
        return 'gaming'
    else:
        return 'general'

df['Laptop purpose']=df['name'].apply(extract_purpose)

## Number of USB 2.0 Ports

In [72]:
df['Number of USB 2.0 Ports']=nan_1_time_cate(df['Number of USB 2.0 Ports'])

## Optical Drive Type

In [73]:
df['Optical Drive Type'] = df['Optical Drive Type'].replace(regex=['no.*', '^(?!no).*'], value=['no', 'yes'])

# last step

In [74]:
# check the number of acutal values in per column
cols=df.columns
remain_df=[]
for col in cols:
    remain=df.shape[0]-df[col].isna().sum()
    remain_df.append([col, remain, df[col].sample(3)])
remain_df.sort(reverse=True, key=lambda x: x[1])
remain_df

[['ASIN',
  2106,
  170     b0b23nrh13
  1158    b0b8zs43fc
  1155    b09rnc69k8
  Name: ASIN, dtype: object],
 ['Item Weight',
  2106,
  432     3.52 pounds
  537     2.47 pounds
  1233     3.3 pounds
  Name: Item Weight, dtype: object],
 ['link',
  2106,
  1962    https://www.amazon.com/sspa/click?ie=utf8&spc=...
  594     https://www.amazon.com/sspa/click?ie=utf8&spc=...
  2039    https://www.amazon.com/microsoft-surface-lapto...
  Name: link, dtype: object],
 ['name',
  2106,
  512    2022 newest hp laptop with microsoft office in...
  168    asus chromebook c403 rugged & spill resistant ...
  156    msi bravo 15 15.6" 144hz gaming laptop: amd ry...
  Name: name, dtype: object],
 ['official link',
  2106,
  340     https://www.amazon.com/hp-15inch-laptop-dual-c...
  1451    https://www.amazon.com/lenovo-slim-7i-windows-...
  680     https://www.amazon.com/msi-summit-business-pro...
  Name: official link, dtype: object],
 ['weight',
  2106,
  822     2.376824
  2081    3.107108
  11

In [75]:
cols_to_keep=[
'name',
'Brand',
'Best Sellers Rank',
'weight',
'Laptop type',
'Laptop purpose',
'Screen Size',
'Hard Drive Size',
'Hard Drive Type',
'Memory Speed',
'Memory Size',
'Memory Type',
'Processor',
'Processor rank',
'Graphics Coprocessor',
'Graphics Coprocessor perf',
'Optical Drive Type',
'Operating System',
'Number of USB 3.0 Ports',
'Number of USB 2.0 Ports',
'Date First Available',
'Customer Reviews',
'old_price',
'price',
]
df=df[cols_to_keep]

In [76]:
for col in df.columns:
    try:
        df[col] = df[col].astype(np.float64)
    except Exception:
        continue

In [81]:
df['Optical Drive Type']

401     NaN
318     yes
121      no
575      no
10       no
       ... 
1755    NaN
1756    NaN
1758    NaN
1795     no
1810     no
Name: Optical Drive Type, Length: 1513, dtype: object

In [78]:
df=df.drop_duplicates(subset=[
'Brand',
'weight',
'Laptop type',
'Laptop purpose',
'Screen Size',
'Hard Drive Size',
'Hard Drive Type',
'Memory Speed',
'Memory Size',
'Memory Type',
'Processor',
'Processor rank',
'Graphics Coprocessor',
'Graphics Coprocessor perf',
'Optical Drive Type',
'Operating System',
'Number of USB 3.0 Ports',
'Number of USB 2.0 Ports',
])

In [79]:
df=df.sort_values(by=['price'])

In [80]:
df.to_csv('temp_final_data.csv', index=False)